### Sources  
* https://medium.com/@traffordDataLab/lets-make-a-map-in-r-7bd1d9366098  
* https://stackoverflow.com/questions/50859765/chloropleth-map-with-geojson-and-ggplot2  
* https://www.r-graph-gallery.com/327-chloropleth-map-from-geojson-with-ggplot2.html  


In [1]:
library(sf)
library(ggplot2)
library(dplyr)
library(viridis)
library(tidyr)
library(gghighlight)
library(plotly)
library(tidyverse)

Linking to GEOS 3.7.1, GDAL 2.4.2, PROJ 5.2.0

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: viridisLite

Attaching package: ‘plotly’

The following object is masked from ‘package:ggplot2’:

    last_plot

The following object is masked from ‘package:stats’:

    filter

The following object is masked from ‘package:graphics’:

    layout

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ tibble  2.1.3     ✔ purrr   0.3.3
✔ readr   1.3.1     ✔ stringr 1.4.0
✔ tibble  2.1.3     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ plotly::filter() masks dplyr::filter(), stats::filter()
✖ dplyr::lag()     masks stats::lag()


In [2]:
# Load map data
cp_map <- st_read("../data/central_park_geo.geojson", quiet = TRUE)

In [3]:
# Update values for different zones of CP West
cp_map$location <- as.character(cp_map$location)
cp_map$sitename <- as.character(cp_map$sitename)
cp_map$sitename[cp_map$location == 'CPW, W 97 St, West Drive, W 100 St'] <- 'Central Park West (Zone 1)'
cp_map$sitename[cp_map$location == 'CPW, 85 St Transverse, West Drive To 96 St'] <- 'Central Park West (Zone 2)'
cp_map$sitename[cp_map$location == 'West Drive, CPW, 65 St Transverse'] <- 'Central Park West (Zone 3)'
cp_map$sitename[cp_map$location == '66 St To 72 St, CPW To West Drive'] <- 'Central Park West (Zone 4)'

# Add numbers for each zone to display on map 


In [4]:
# Read in squirrel data
squirrel_data <- read.csv('../data/squirrel_data.csv')

In [5]:
# Filter data for counts about 5 squirrels
gt_5_sq <- squirrel_data %>% select(sitename, Unique_Squirrel_ID) %>% filter(Unique_Squirrel_ID > 5) %>% pull(sitename)

In [81]:
# Find center of each polygon to plot region number
centroids_filtered <- cp_map %>%
    filter((sitename %in% gt_5_sq)) %>%
    st_centroid() %>% 
    bind_cols(as_data_frame(st_coordinates(.))) %>%
    # Add column for number to plot on choropleth
    mutate(number = seq(1:40))

Warning message in st_centroid.sf(.):
“st_centroid assumes attributes are constant over geometries of x”Warning message in st_centroid.sfc(st_geometry(x), of_largest_polygon = of_largest_polygon):
“st_centroid does not give correct centroids for longitude/latitude data”

In [82]:
#Join map boundaries to squirrel data
data_map <- left_join(cp_map %>% filter(sitename %in% gt_5_sq), squirrel_data, by = 'sitename')

Warning message:
“Column `sitename` joining character vector and factor, coercing into character vector”

In [83]:
# Add column with umber that corresponds to number in centroids_filtered
data_map$number <- seq(1:nrow(data_map))

In [86]:
# Change colname for behaviors
colnames(data_map) <- c('propname',
'us_congres',
'zipcode',
'acres',
'location',
'gispropnum',
'retired',
'subcategor',
'communityb',
'department',
'precinct',
'retireddat',
'omppropid',
'nys_assemb',
'sitename',
'nys_senate',
'councildis',
'borough',
'descriptio',
'X',
'Unique_Squirrel_ID',
'Climbing',
'Approaching_humans',
'Vocalizing',
'Running_or_chasing',
'Eating_or_foraging',
'Count_diff',
'geometry',
'number')

In [90]:
# Add column to plot color difference
data_map <- data_map %>% 
    mutate(am_pm = case_when(Count_diff < 0 ~ "Higher count in PM", 
                            TRUE ~ "Higher count in AM"))

In [129]:
# Font size variables for below plots
ys = 14
xs = 14
ts = 24

In [ ]:
#' Plots a choropleth of central park filled with color
#' according to count of squirrels. 
#'
#' @param highlight (optional) a character vector of sitenames to highlight
#'
#' @return ggplot chart
#'
#' @examples
#' plot_choropleth(c('Ross Pinetum', 'The Ramble'))
plot_choropleth <- function(highlight = vector()) {
    full_map <- ggplot(data_map) + 
        geom_sf(aes(fill = Unique_Squirrel_ID), color = 'lightgrey') + 
        geom_text(aes(X, Y, label = number), 
                  data = centroids_filtered, 
                  size = 3, 
                  color = 'black') +        
        scale_fill_gradient(low = 'white', high = 'darkgreen', 
                            limits = c(0,max(squirrel_data$Unique_Squirrel_ID)),
                            name = 'Count') +
        labs(title = 'Central Park Squirrel Distribution', x = '', y = '') +
        theme_minimal() +
        theme(legend.position = c(0.8, 0.2), 
        plot.title = element_text(hjust = 0.5, size = ts), 
        axis.text.x = element_text(angle = 90, hjust = 1, size = xs),
        axis.text.y = element_text(size = ys))
    if (length(highlight) == 0) {
        ggplotly(full_map)
    } else {
        m_h <- full_map + 
        gghighlight(sitename %in% highlight, 
                    label_key = Unique_Squirrel_ID)
        ggplotly(m_h)
    }
}

In [ ]:
#' Plots a bar chart of squirrel count by park region
#'
#' @param highlight (optional) a character vector of sitenames to highlight
#'
#' @return ggplot chart
#'
#' @examples
#' plot_counts_bar(c('Ross Pinetum', 'The Ramble'))
plot_counts_bar <- function(highlight = vector()) {
    counts_full <- ggplot(data_map) + 
            geom_bar(aes(x = reorder(sitename, Unique_Squirrel_ID), 
                         y = Unique_Squirrel_ID, 
                         fill = Unique_Squirrel_ID, text = paste("Area name:", sitename, '<br>',"Squirrel count:", Unique_Squirrel_ID)), 
                     stat = 'identity') + 

            coord_flip() +
            scale_fill_gradient(low = 'white', 
                                high = 'darkgreen', 
                                limits = c(0,max(squirrel_data$Unique_Squirrel_ID)),
                                name = 'Count') +
            labs(title = 'Squirrel Distribution by Park Region', y = 'Squirrel Count', x = '', size = xs) +
            theme_minimal() +
            theme(panel.grid.major.y = element_blank(), 
            legend.position = c(0.8, 0.2), 
            plot.title = element_text(hjust = 0.5, size = ts), 
            axis.text = element_text(hjust = 1, size = xs),
            axis.title = element_text(size = ys))
    if (length(highlight) == 0) {
        ggplotly(counts_full, tooltip="text")
        } else {
            c_h <- counts_full +
                 gghighlight(sitename %in% highlight, 
                             label_key = Unique_Squirrel_ID)
            ggplotly(c_h,tooltip=c("x","y"))
    }
}

In [ ]:
#' Plots a bar chart of squirrel behavior by park region
#'
#' @param highlight (default = empty) a character vector of sitenames to highlight
#' @param behavior (default = Running_or_chasing) column of behavior to plot with options
#'       Running_or_chasing, Eating_or_foraging, Vocalizing, Approaching_humans, Climbing
#'
#' @return ggplot chart
#'
#' @examples
#' plot_behaviors_bar(c('Ross Pinetum', 'The Ramble'))
plot_behaviors_bar <- function(behavior = 'Running_or_chasing', highlight = vector()) {
    b_bar <- ggplot(data_map) + 
        geom_bar(aes(reorder(sitename, !!sym(behavior)), !!sym(behavior),text = paste("Area name:", sitename, '<br>',"Squirrel count:", !!sym(behavior))),
                     stat = 'identity') + 
        coord_flip() +
        labs(title = paste('Count of Squirrels',
                           str_replace_all(behavior,'_',' '),
                           ' by Park Region'), 
             y = 'Squirrel Count', x = '') +
        theme_minimal() +
        theme(panel.grid.major.y = element_blank(), 
            plot.title = element_text(hjust = 0.5, size = ts), 
            axis.text = element_text(hjust = 1, size = xs),
            axis.title = element_text(size = ys))
    if (length(highlight) == 0) {
        ggplotly(b_bar, tooltip="text")
    } else {
        b_h <- b_bar + gghighlight(sitename %in% highlight, 
            label_key = Unique_Squirrel_ID)
        ggplotly(b_h,tooltip=c("x","y"))
    }
}


In [128]:
# Use a function make_graph() to create the graph

#' Plots a bar chart of change in squirrel count from AM to PM
#'    by park region
#'
#' @param highlight (optional) a character vector of sitenames to highlight
#'
#' @return ggplot chart
#'
#' @examples
#' plot_diff_bar(c('Ross Pinetum', 'The Ramble'))
plot_diff_bar <- function(highlight = list()) {
    if (length(highlight) == 0){
      diff_bar <- ggplot(data_map,
               aes(x = reorder(sitename, -Count_diff), 
                   y = -Count_diff,
                   fill = am_pm, text = paste("Area name:", sitename, '<br>',"Count difference:", -Count_diff))) +
        geom_bar(stat = "identity")  +
        labs(title = 'Squirrel Abundance: Morning vs. Afternoon', 
             y = 'Difference in count', 
             x = '') +
        scale_y_continuous(breaks=seq(-20,50,10)) +
        coord_flip() +
        theme_minimal() +
        theme(panel.grid.major.y = element_blank(), 
              legend.position = c(0.8, 0.1), 
              plot.title = element_text(hjust = 0.5, size = ts), 
              axis.text = element_text(hjust = 1, size = xs),
              axis.title = element_text(size = ys),
              legend.title = element_blank()) 

      ggplotly(diff_bar, tooltip="text")
    } else {
        diff_bar <- ggplot(data_map,
               aes(x = reorder(sitename, -Count_diff), 
                   y = -Count_diff,
                   fill = am_pm)) +
        geom_bar(stat = "identity")  +
        labs(title = 'Squirrel Abundance: Morning vs. Afternoon', 
             y = 'Difference in count', 
             x = '') +
        scale_y_continuous(breaks=seq(-20,50,10)) +
        coord_flip() +
        theme_minimal() +
        theme(panel.grid.major.y = element_blank(), 
              legend.position = c(0.8, 0.1), 
              plot.title = element_text(hjust = 0.5, size = ts), 
              axis.text = element_text(hjust = 1, size = xs),
              axis.title = element_text(size = ys),
              legend.title = element_blank()) 

        db_h <- diff_bar + gghighlight(sitename %in% highlight, 
            label_key = Unique_Squirrel_ID)
        ggplotly(db_h, tooltip=c("x","y"))
    }

}